<a href="https://colab.research.google.com/github/Fuenfgeld/ATeamDatenmanagementUndArchivierung/blob/lknegendorf-sql_statement/IMECOS_spielwiese_felix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# import libraries
import sqlite3 as sq
from sqlite3 import Error
import pandas as pd
import numpy as np
import requests


In [17]:
#mount drive to access data
from google.colab import drive 
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [18]:
# SQLite connection
def create_connection_file():
    """ create a database connection to a database that resides
        in a file(db_file/imecos.db)
    """
    conn = None;
    try:
       #Establishing the connection
        conn = sq.connect('/content/gdrive/Shareddrives/IMECOS/imecos.db')
        return conn
        print(sqlite3.version)
    except Error as e:
        print("Error while connecting to sqlite", e)
    # finally:
    #     if conn:
    #         conn.close() 
    # Creating a temporary empty database in the memory.
conn = create_connection_file()
# Creating a cursor object using the cursor() method
cur = conn.cursor()
print("Successfully Connected to SQLite")

Successfully Connected to SQLite


In [ ]:
# material_path = "/content/drive/Shareddrives/05_BIDS_Datenmanagement & Archivierung im Umfeld der Forschung/Material"

In [ ]:
# !ls "/content/drive/Shareddrives/05_BIDS_Datenmanagement & Archivierung im Umfeld der Forschung/Material/source_dbs"

In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [ ]:
df = pd.read_csv('/content/gdrive/Shareddrives/IMECOS/csv_data-20210115T203102Z-001.zip (Unzipped Files)/csv_data/allergy/careplans.csv') 
#print(df.head)
print(df.loc[0])

Id                   b066696b-4a2f-ec29-b018-bcf7464d020c
START                                          1943-02-15
STOP                                                  NaN
PATIENT              3575b903-dbd0-1d55-6146-9e8aa4ed52a5
ENCOUNTER            fd59905a-4ec8-62a6-d45d-d2a458d60111
CODE                                            384758001
DESCRIPTION           Self-care interventions (procedure)
REASONCODE                                            NaN
REASONDESCRIPTION                                     NaN
Name: 0, dtype: object


In [ ]:
cursor = conn.cursor()
#cursor.execute('drop table people_info')
#cursor.execute('CREATE TABLE people_info (Name nvarchar(50), Country nvarchar(50), Age int)')
cursor.execute('INSERT INTO people_info VALUES (12, 13, 14)')
cursor.execute('select * from people_info')

In [24]:
sql_create_source_data = requests.get('https://raw.githubusercontent.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung/lknegendorf-sql_statement/create_statements_source_db.sql').text
cursor = conn.cursor()
cursor.executescript(sql_create_source_data)
conn.commit()


In [25]:
cursor.execute("SELECT name FROM sqlite_master WHERE type = 'table'")
print(cursor.fetchall())

[('patients',), ('encounters',), ('tb_patients',), ('tb_organizations',), ('tb_providers',), ('tb_payers',), ('tb_payer_transitions',), ('tb_encounters',), ('tb_careplans',), ('tb_conditions',), ('tb_devices',), ('tb_disease',), ('tb_imaging_studies',), ('tb_immunizations',), ('tb_observations',), ('tb_procedures',), ('tb_supplies',), ('tb_medications',)]


In [ ]:
cursor = conn.cursor()
cursor.execute('CREATE TABLE patients (
PATIENT_ID varchar(256) primary key,
#BIRTHDATE date,
#DEATHDATE date
#SSN varchar(256),
#DRIVERS varchar(256),
#PASSPORT varchar(256),
#PREFIX varchar(256),
#FIRST varchar(256),
#LAST varchar(256),
#SUFFIX varchar(256),
#MAIDEN varchar(256),
#MARITAL char(1),
#RACE varchar(256),
#ETHNICITY varchar(256),
#GENDER char(1),
#BIRTHPLACE varchar(256),
#ADDRESS varchar(256),
#CITY varchar(256),
#STATE varchar(256),
#COUNTY varchar(256),
#ZIP varchar(256),
#LAT numeric(8,6),
#LON numeric(8,6),
#HEALTHCARE_EXPENSES numeric(10,2),
#HEALTHCARE_COVERAGE numeric(10,2), 
#DATASET_ORIGIN varchar(256)
  )
')

SyntaxError: ignored

In [ ]:
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO TestDB.dbo.people_info (Name, Country, Age)
                VALUES (?,?,?)
                ''',
                row.Name, 
                row.Country,
                row.Age
                )
conn.commit()